In [1]:
import warnings
warnings.filterwarnings('ignore')
import os

import tws_data 
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm
from ib_insync import *
import numpy as np
import pandas as pd
import datetime as dt
import time
pd.options.display.float_format = '{:,.4f}'.format
%matplotlib inline
from scipy.stats import norm as norm
import matplotlib.pyplot as plt
import requests as req
from bs4 import BeautifulSoup as bs
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import math


In [2]:
os.chdir('D:\Options Data\IB Intraday')

def create_dfrow(curr_option):
    df_row = pd.DataFrame({'Symbol': curr_option.contract.localSymbol, 
                           'Type':curr_option.contract.right, 
                           'Bid': curr_option.bid, 
                           'Ask': curr_option.ask, 
                           'Volume': curr_option.volume, 
                           'Strike': curr_option.contract.strike,
                           'Expiry': dt.datetime.strptime(curr_option.contract.lastTradeDateOrContractMonth, '%Y%m%d'),
                           'bidIV': curr_option.bidGreeks.impliedVol, 
                           'bidDelta': curr_option.bidGreeks.delta,
                           'bidGamma': curr_option.bidGreeks.gamma, 
                           'bidVega': curr_option.bidGreeks.vega,
                           'bidTheta': curr_option.bidGreeks.theta, 
                           'askIV': curr_option.askGreeks.impliedVol, 
                           'askDelta': curr_option.askGreeks.delta,
                           'askGamma': curr_option.askGreeks.gamma, 
                           'askVega': curr_option.askGreeks.vega,
                           'askTheta': curr_option.askGreeks.theta,
                           'modelIV': curr_option.modelGreeks.impliedVol, 
                           'modelDelta': curr_option.modelGreeks.delta,
                           'modelGamma': curr_option.modelGreeks.gamma, 
                           'modelVega': curr_option.modelGreeks.vega,
                           'modelTheta': curr_option.modelGreeks.theta}, index = [0])
    return df_row

def get_movers():

    indexarb_url = 'http://www.indexarb.com/stockVsIndexesMiddleColumn.html#djIndexAnalysis'
    indexarb_soup = bs(req.get(indexarb_url).text,'lxml')
    indexarb_tables = indexarb_soup.select('table[align="LEFT"]')

    movers = []

    for i, table_html in enumerate(indexarb_tables):
        curr_table = pd.read_html(str(table_html))[2]
        curr_table.columns = curr_table.loc[1,:].tolist()

        if i == 0:
            curr_table = curr_table.iloc[2:,:4].set_index('Index').apply(pd.to_numeric)
        else:
            curr_table = curr_table.iloc[2:,[1,3,4,5,6]].dropna().set_index('Sym').apply(pd.to_numeric)

        curr_table.columns = [x.split(' ')[0] for x in curr_table.columns.tolist()]
        movers.append(curr_table)

    index = movers[0]
    movers = pd.concat(movers[1:], axis = 0)
    movers = movers.sort_values('OneMonth')
    
    return index, movers

def time_remaining(weekly_options, curr_spx):
    typ = weekly_options.reset_index().loc[0,'Type']

    try:
        interest_rate = bs(req.get('https://www.marketwatch.com/investing/bond/tmubmusd01m?countrycode=bx','lxml').text, 'lxml')
        interest_rate = float(interest_rate.select_one('h3[class*="intraday__price"]').select_one('bg-quote').text)/100
    except:
        interest_rate = 0.02422

    weekly_options['a'] = interest_rate - (weekly_options['modelIV']**2)/2

    if typ == 'P':
        weekly_options['b'] = weekly_options['modelIV']*norm.ppf(-weekly_options['modelDelta'])
    if typ == 'C':
        weekly_options['b'] = -weekly_options['modelIV']*norm.ppf(weekly_options['modelDelta'])
    weekly_options['c'] = np.log(curr_spx/weekly_options['Strike'])

    weekly_options_otm = weekly_options[abs(weekly_options.modelDelta) <= 0.5]
    weekly_options_itm = weekly_options[abs(weekly_options.modelDelta) > 0.5]

    if typ == 'P':
        weekly_options_otm['time_remaining'] = ((-weekly_options_otm.b - 
                                                 np.sqrt(weekly_options_otm.b**2 - 
                                                         4*weekly_options_otm.a*weekly_options_otm.c))/(2*weekly_options_otm.a))**2
        weekly_options_itm['time_remaining'] = ((-weekly_options_itm.b + 
                                                 np.sqrt(weekly_options_itm.b**2 - 
                                                         4*weekly_options_itm.a*weekly_options_itm.c))/(2*weekly_options_itm.a))**2
    if typ == 'C':
        weekly_options_otm['time_remaining'] = ((-weekly_options_otm.b + 
                                                 np.sqrt(weekly_options_otm.b**2 - 
                                                         4*weekly_options_otm.a*weekly_options_otm.c))/(2*weekly_options_otm.a))**2
        weekly_options_itm['time_remaining'] = ((-weekly_options_itm.b - 
                                                 np.sqrt(weekly_options_itm.b**2 - 
                                                         4*weekly_options_itm.a*weekly_options_itm.c))/(2*weekly_options_itm.a))**2
    weekly_times = pd.concat([weekly_options_otm, 
                              weekly_options_itm], axis = 0)[['Expiry','time_remaining']].dropna().groupby('Expiry').mean()

    del weekly_options['a'], weekly_options['b'], weekly_options['c'], weekly_options_itm, weekly_options_otm

    return weekly_options.merge(weekly_times.reset_index(), on = 'Expiry')


def get_options(ticker, exchange, contractType, earliest_date, latest_date, curr_iv, stdevs, save_csv = False):
    if contractType == 'Stock':
        security = Stock(ticker,exchange)
    else:
        security = Index(ticker, exchange)

    ib.qualifyContracts(security)

    chains = ib.reqSecDefOptParams(security.symbol, '', security.secType, security.conId)

    chains_df = util.df(chains)
    chain = next(c for c in chains if c.tradingClass == ticker and c.exchange == 'SMART')

    all_expirations = sorted(exp for exp in chain.expirations)
    expirations_dates = [dt.datetime.strptime(x, '%Y%m%d') for x in all_expirations]

    expirations = list(filter(lambda x: x[0] >= earliest_date and x[0] <= latest_date, list(zip(expirations_dates,all_expirations))))
    dte_expirations = [(expir[0] - dt.datetime.today()).days for expir in expirations]
    expirations = [expir[1] for expir in expirations]

    ib.reqHeadTimeStamp(security, whatToShow='TRADES', useRTH=True)

    end_date = '' #'20100506 13:00:00 PST'
    duration = '1 D'

    sec_bars = ib.reqHistoricalData(
            security,
            endDateTime=end_date,
            durationStr=duration,
            barSizeSetting='1 min',
            whatToShow='TRADES',
            useRTH=True,
            formatDate=1)


    sec_df = util.df(sec_bars).set_index('date')
    sec_df.index = pd.to_datetime(sec_df.index)

    curr_sec_price = sec_df.reset_index().loc[len(sec_df) - 1, 'close']
    bounds_lst = [curr_sec_price*(stdevs*stdevs/(100*np.sqrt(365.25/dte))) for dte in dte_expirations]


    contracts_list = []

    for bounds, expiration in zip(bounds_lst, expirations):
#         strikes = [strike for strike in chain.strikes
#                    if strike % 5 == 0
#                    and curr_sec_price - bounds < strike < curr_sec_price + bounds]
        strikes = [strike for strike in chain.strikes
                   if curr_sec_price - bounds < strike < curr_sec_price + bounds]

        rights = ['P', 'C']

        curr_contracts = [Option(ticker, expiration, strike, right, 'SMART')
                          for right in rights
                          for strike in strikes]
        contracts_list += [curr_contracts]

    puts_list = []
    calls_list = []

    for contracts in contracts_list:
        ib.qualifyContracts(*contracts)
        sec_options = ib.reqTickers(*contracts)

        puts_rows = []
        calls_rows = []

        for curr_option in sec_options:
            try:
                curr_row = create_dfrow(curr_option)
                if curr_option.contract.right == 'P':
                    puts_rows.append(curr_row)
                elif curr_option.contract.right == 'C':
                    calls_rows.append(curr_row)
            except:
                continue

        puts = pd.concat(puts_rows).reset_index(drop = True).sort_values(['Expiry','Strike'])
        calls = pd.concat(calls_rows).reset_index(drop = True).sort_values(['Expiry','Strike'])
        puts = time_remaining(puts, curr_sec_price)
        calls = time_remaining(calls, curr_sec_price)
        puts_list.append(puts)
        calls_list.append(calls)
        time.sleep(5)

    puts_df = pd.concat(puts_list,axis = 0).reset_index(drop = True)
    calls_df = pd.concat(calls_list,axis = 0).reset_index(drop = True)

    if save_csv:
        curr_time = dt.datetime.now().strftime('%Y%m%d-%H-%M')
        puts_df.to_csv('ib_puts' + curr_time + '.csv')
        calls_df.to_csv('ib_calls' + curr_time + '.csv')
        
    return puts_df, calls_df, curr_sec_price


def get_spreads(weekly_options, curr_spx, expiry_index = 0, rowshift = 1, commissions = 0.0266):

    expirations_dates = weekly_options.Expiry.drop_duplicates().tolist()

    weekly_options = weekly_options[weekly_options.Expiry == expirations_dates[expiry_index]]

    typ = weekly_options.reset_index().loc[0,'Type']

    if typ == 'P':
        weekly_options = weekly_options.sort_values('Strike', ascending = False).reset_index(drop = True)
    if typ == 'C':
        weekly_options = weekly_options.sort_values('Strike', ascending = True).reset_index(drop = True)

    time_remaining = weekly_options.reset_index(drop = True).loc[0,'time_remaining']

    weekly_options['Mid'] = (weekly_options.Ask + weekly_options.Bid)/2
    shorts = weekly_options[['Mid', 'Strike', 'bidDelta', 'bidGamma', 'bidTheta', 'bidVega']]
    shorts.columns = ['Price','Strike','Delta','Gamma','Theta','Vega']
    longs = weekly_options[['Mid','Strike', 'askDelta', 'askGamma','askTheta', 'askVega']]
    longs.columns = ['Price','Strike','Delta','Gamma','Theta','Vega']

    spreads = shorts - longs.shift(-rowshift)
    spreads['Short_Strike'] = shorts.Strike
    spreads['Short_Delta'] = shorts.Delta
    spreads['Long_Strike'] = longs.Strike.shift(-rowshift)
    spreads = spreads[['Short_Strike', 'Long_Strike', 'Short_Delta', 'Price',
                       'Strike', 'Delta', 'Gamma', 'Theta', 'Vega']]

    spreads['Credit'] = spreads.Price - commissions

    if typ == 'P':
        spreads['MaxLoss'] = (-spreads.Strike + spreads.Credit)*100
        spreads['BreakEven'] = spreads['Short_Strike'] - spreads.Credit
    if typ == 'C':
        spreads['MaxLoss'] = (spreads.Strike + spreads.Credit)*100
        spreads['BreakEven'] = spreads['Short_Strike'] + spreads.Credit

    del spreads['Price'], spreads['Strike']

    spreads = spreads[spreads.Credit > 0].dropna().reset_index(drop = True)
    spreads['Expiry'] = expirations_dates[expiry_index]

    return spreads

def art_hedges(curr_calls, curr_puts, curr_price, expiry_index, bias = 'upside',
               maxshift = 6, commissions = 0.0266):
    
    hedge = []
    for rowshift in range(1,maxshift):

        if bias == 'upside':
            spreads = get_spreads(curr_puts, curr_price, expiry_index, rowshift, commissions = 0.0266)
        else:
            spreads = get_spreads(curr_calls, curr_price, expiry_index, rowshift, commissions = 0.0266)
        spreads = spreads[(spreads.Credit > 0.5)]
        hedge.append(spreads)

    hedge = pd.concat(hedge, axis = 0).reset_index(drop = True)

    if bias == 'upside':
        expirations_dates = curr_calls.Expiry.drop_duplicates().tolist()

        long_hedge = curr_calls[(curr_calls.Expiry == expirations_dates[expiry_index])][['Ask', 'Bid', 'Expiry', 'Strike',
                                                                             'modelDelta', 'modelGamma',
                                                                             'modelTheta', 'modelVega', 'time_remaining']].reset_index(drop = True)
    else:
        expirations_dates = curr_puts.Expiry.drop_duplicates().tolist()

        long_hedge = curr_puts[(curr_puts.Expiry == expirations_dates[expiry_index])][['Ask', 'Bid', 'Expiry', 'Strike',
                                                                             'modelDelta', 'modelGamma',
                                                                             'modelTheta', 'modelVega', 'time_remaining']].reset_index(drop = True)

    long_hedge['Mid'] = (long_hedge['Ask'] + long_hedge['Bid'])/2
    long_hedge.columns = [x.replace('model','') for x in long_hedge.columns.tolist()]

    hedges_lst = []

    for idx, row in hedge.iterrows():

        curr_hedge = long_hedge[long_hedge.Mid < row.Credit][['Mid','Expiry', 'Strike',
                                                              'Delta', 'Gamma',
                                                              'Theta', 'Vega', 'time_remaining']]
        curr_hedge['Gamma'] = curr_hedge['Gamma'] - row.Gamma
        curr_hedge['Delta'] = curr_hedge['Delta'] - row.Delta
        curr_hedge['Theta'] = curr_hedge['Theta'] - row.Theta
        curr_hedge['Vega'] = curr_hedge['Vega'] - row.Vega
        if bias == 'upside':
            curr_hedge = curr_hedge[(curr_hedge.Delta > 0) &
                                    (curr_hedge.Gamma > 0) &
                                    (curr_hedge.Theta > 0) &
                                    (curr_hedge.Vega > 0)]
        else:
            curr_hedge = curr_hedge[(curr_hedge.Delta < 0) &
                                (curr_hedge.Gamma > 0) &
                                (curr_hedge.Theta > 0) &
                                (curr_hedge.Vega > 0)]
        curr_hedge['Credit'] = -curr_hedge.Mid + row.Credit - 0.0133

        if bias == 'upside':
            curr_hedge['MaxLoss'] = (row.Long_Strike - row.Short_Strike + curr_hedge.Credit)*100
            curr_hedge['BreakEven'] = row.Short_Strike - curr_hedge['Credit']
        else:
            curr_hedge['MaxLoss'] = (row.Short_Strike - row.Long_Strike + curr_hedge.Credit)*100
            curr_hedge['BreakEven'] = row.Short_Strike + curr_hedge['Credit']
        curr_hedge['Maturity'] = curr_hedge['Expiry']
        curr_hedge['Mid'] = row.Short_Strike
        curr_hedge['Expiry'] = row.Long_Strike

        if bias == 'upside':
            curr_hedge.columns = ['Short_Put_Strike','Long_Put_Strike','Long_Call_Strike'] + curr_hedge.columns.tolist()[3:]
        else:
            curr_hedge.columns = ['Short_Call_Strike','Long_Call_Strike','Long_Put_Strike'] + curr_hedge.columns.tolist()[3:]

        hedges_lst.append(curr_hedge)

    hedges_df = pd.concat(hedges_lst, axis = 0)
    
    return hedges_df

In [3]:
index, movers = get_movers()

In [4]:
losers = movers.head()
winners = movers.tail().sort_values('OneMonth', ascending = False)

In [3]:
client_id = 12

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=client_id)

<IB connected to 127.0.0.1:7496 clientId=12>

In [4]:
# exchange = 'NYSE'
# contractType = 'Stock'
# earliest_date = dt.datetime(2019,4,6)
# latest_date = dt.datetime(2019,6,30)
# curr_iv = 40
# stdevs = 9

# stock_prices = {}
# stock_puts = {}
# stock_calls = {}

# for ticker in losers.index:

#     curr_puts, curr_calls, curr_price = get_options(ticker, exchange, contractType, earliest_date, latest_date, curr_iv, stdevs, save_csv = False)
    
#     stock_prices[ticker] = curr_price
#     stock_puts[ticker] = curr_puts
#     stock_calls[ticker] = curr_calls
#     break

In [5]:
exchange = 'NYSE'
contractType = 'Stock'
earliest_date = dt.datetime(2020,9,1)
latest_date = dt.datetime(2020,12,1)

ticker = 'GOOGL'
curr_iv = 80
stdevs = 30
curr_puts, curr_calls, curr_price = get_options(ticker, exchange, contractType, earliest_date, latest_date, curr_iv, stdevs, save_csv = False)

Started to throttle requests
Error 200, reqId 9: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1025.0, right='P', exchange='SMART')
Error 200, reqId 10: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1537.5, right='P', exchange='SMART')
Error 200, reqId 11: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1527.5, right='P', exchange='SMART')
Error 200, reqId 14: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1030.0, right='P', exchange='SMART')
Error 200, reqId 13: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=2050.0, right='P', exchange='SMART')
Er

Error 200, reqId 61: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=600.0, right='P', exchange='SMART')
Error 200, reqId 63: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1115.0, right='P', exchange='SMART')
Error 200, reqId 64: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1630.0, right='P', exchange='SMART')
Error 200, reqId 66: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1635.0, right='P', exchange='SMART')
Error 200, reqId 67: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1125.0, right='P', exchange='SMART')
Error 200, reqId 68: No securit

Error 200, reqId 117: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1230.0, right='P', exchange='SMART')
Error 200, reqId 118: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=720.0, right='P', exchange='SMART')
Error 200, reqId 119: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1745.0, right='P', exchange='SMART')
Error 200, reqId 120: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1235.0, right='P', exchange='SMART')
Error 200, reqId 123: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1755.0, right='P', exchange='SMART')
Error 200, reqId 121: No s

Error 200, reqId 173: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1352.5, right='P', exchange='SMART')
Error 200, reqId 176: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1870.0, right='P', exchange='SMART')
Error 200, reqId 178: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1362.5, right='P', exchange='SMART')
Error 200, reqId 180: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1367.5, right='P', exchange='SMART')
Error 200, reqId 184: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1372.5, right='P', exchange='SMART')
Error 200, reqId 186: No 

Error 200, reqId 260: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1497.5, right='P', exchange='SMART')
Error 200, reqId 263: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1502.5, right='P', exchange='SMART')
Error 200, reqId 262: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=990.0, right='P', exchange='SMART')
Error 200, reqId 266: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1507.5, right='P', exchange='SMART')
Error 200, reqId 267: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=2020.0, right='P', exchange='SMART')
Error 200, reqId 265: No s

...

Error 200, reqId 784: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=950.0, right='P', exchange='SMART')
Error 200, reqId 785: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1462.5, right='P', exchange='SMART')
Error 200, reqId 787: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=955.0, right='P', exchange='SMART')
Error 200, reqId 788: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1467.5, right='P', exchange='SMART')
Error 200, reqId 792: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1472.5, right='P', exchange='SMART')
Error 200, reqId 794: No se

Error 200, reqId 852: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1532.5, right='C', exchange='SMART')
Error 200, reqId 853: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1565.0, right='C', exchange='SMART')
Error 200, reqId 854: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1055.0, right='C', exchange='SMART')
Error 200, reqId 855: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=2080.0, right='C', exchange='SMART')
Error 200, reqId 856: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=545.0, right='C', exchange='SMART')
Error 200, reqId 857: No s

Error 200, reqId 905: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1670.0, right='C', exchange='SMART')
Error 200, reqId 907: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1675.0, right='C', exchange='SMART')
Error 200, reqId 908: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1165.0, right='C', exchange='SMART')
Error 200, reqId 910: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1170.0, right='C', exchange='SMART')
Error 200, reqId 911: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=660.0, right='C', exchange='SMART')
Error 200, reqId 912: No s

Error 200, reqId 962: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1290.0, right='C', exchange='SMART')
Error 200, reqId 963: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=780.0, right='C', exchange='SMART')
Error 200, reqId 964: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1295.0, right='C', exchange='SMART')
Error 200, reqId 965: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1810.0, right='C', exchange='SMART')
Error 200, reqId 967: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1302.5, right='C', exchange='SMART')
Error 200, reqId 968: No s

Error 200, reqId 1032: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=910.0, right='C', exchange='SMART')
Error 200, reqId 1033: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1422.5, right='C', exchange='SMART')
Error 200, reqId 1035: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=915.0, right='C', exchange='SMART')
Error 200, reqId 1036: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1427.5, right='C', exchange='SMART')
Error 200, reqId 1040: No security definition has been found for the request, contract: Option(symbol='GOOGL', lastTradeDateOrContractMonth='20200918', strike=1432.5, right='C', exchange='SMART')
Error 200, reqId 1042:

In [6]:
maxshift = 50

arteHedges_upside = []
for expiry_index in range(len(curr_calls.Expiry.drop_duplicates())):
    bias = 'upside'
    possible_pos = art_hedges(curr_calls, curr_puts, curr_price, expiry_index, bias,
                              maxshift = maxshift, commissions = 0.0266)
    arteHedges_upside.append(possible_pos)

arteHedges_upside = pd.concat(arteHedges_upside,axis = 0)

In [7]:
arteHedges_downside = []
for expiry_index in range(len(curr_calls.Expiry.drop_duplicates())):
    bias = 'downside'
    possible_pos = art_hedges(curr_calls, curr_puts, curr_price, expiry_index, bias,
                              maxshift = maxshift, commissions = 0.0266)
    arteHedges_downside.append(possible_pos)
arteHedges_downside = pd.concat(arteHedges_downside,axis = 0)

arteHedges_upside['ReturnOnRisk'] = abs(arteHedges_upside.Credit/arteHedges_upside.MaxLoss)
arteHedges_downside['ReturnOnRisk'] = abs(arteHedges_downside.Credit/arteHedges_downside.MaxLoss)

In [8]:
potential_ups = arteHedges_upside[(arteHedges_upside.Short_Put_Strike < curr_price) &
                                  (arteHedges_upside.Short_Put_Strike > 0.001)].sort_values(['Maturity', 
                                                                                             'ReturnOnRisk'], ascending = False)
try:
    potential_ups['downmove_to_breach'] = potential_ups['Short_Put_Strike']/curr_price - 1
    potential_ups['upmove_to_win'] = potential_ups['Long_Call_Strike']/curr_price - 1
    potential_ups = potential_ups.sort_values('MaxLoss').reset_index(drop = True)
    potential_ups['MaxLossScore'] = potential_ups.index
    
    potential_ups = potential_ups.sort_values('downmove_to_breach', ascending = False).reset_index(drop = True)
    potential_ups['DownMoveScore'] = potential_ups.index
    
    potential_ups = potential_ups.sort_values('upmove_to_win', ascending = False).reset_index(drop = True)
    potential_ups['UpMoveScore'] = potential_ups.index
    
    potential_ups['Score'] = potential_ups.MaxLossScore + potential_ups.DownMoveScore + potential_ups.UpMoveScore
    potential_ups = potential_ups.sort_values('Score', ascending = False)
except:
    None
potential_ups

,Short_Put_Strike,Long_Put_Strike,Long_Call_Strike,Delta,Gamma,Theta,Vega,time_remaining,Credit,MaxLoss,BreakEven,Maturity,ReturnOnRisk,downmove_to_breach,upmove_to_win,MaxLossScore,DownMoveScore,UpMoveScore,Score


In [9]:
potential_downs = arteHedges_downside[(arteHedges_downside.Short_Call_Strike > curr_price)].sort_values(['Maturity', 
                                                                                                         'Long_Put_Strike'], ascending = False)
try:
    potential_downs['upmove_to_breach'] = potential_downs['Short_Call_Strike']/curr_price - 1
    potential_downs['downmove_to_win'] = potential_downs['Long_Put_Strike']/curr_price - 1
    potential_downs = potential_downs.sort_values('MaxLoss').reset_index(drop = True)
    potential_downs['MaxLossScore'] = potential_downs.index
    
    potential_downs = potential_downs.sort_values('upmove_to_breach').reset_index(drop = True)
    potential_downs['UpMoveScore'] = potential_downs.index
    
    potential_downs = potential_downs.sort_values('downmove_to_win').reset_index(drop = True)
    potential_downs['DownMoveScore'] = potential_downs.index
    
    potential_downs['Score'] = potential_downs.MaxLossScore + potential_downs.UpMoveScore + potential_downs.DownMoveScore
    potential_downs = potential_downs.sort_values('Score', ascending = False)
except:
    None
potential_downs

,Short_Call_Strike,Long_Call_Strike,Long_Put_Strike,Delta,Gamma,Theta,Vega,time_remaining,Credit,MaxLoss,BreakEven,Maturity,ReturnOnRisk,upmove_to_breach,downmove_to_win,MaxLossScore,UpMoveScore,DownMoveScore,Score
115,"1,560.0000","1,980.0000","1,350.0000",-0.6386,0.0001,0.0006,0.6714,0.5479,43.1601,"-37,683.9900","1,603.1601",2020-09-18,0.0011,0.0310,-0.1078,93,155,115,363
130,"1,545.0000","1,980.0000","1,355.0000",-0.6672,0.0001,0.0004,0.9408,0.5479,48.8101,"-38,618.9900","1,593.8101",2020-09-18,0.0013,0.0211,-0.1045,85,146,130,361
137,"1,560.0000","2,000.0000","1,360.0000",-0.6551,0.0001,0.0004,0.7509,0.5479,41.4101,"-39,858.9900","1,601.4101",2020-09-18,0.0010,0.0310,-0.1012,59,161,137,357
132,"1,560.0000","2,000.0000","1,355.0000",-0.6490,0.0001,0.0021,0.7398,0.5479,42.7101,"-39,728.9900","1,602.7101",2020-09-18,0.0011,0.0310,-0.1045,63,159,132,354
101,"1,560.0000","1,980.0000","1,345.0000",-0.6335,0.0001,0.0024,0.4816,0.5479,44.3601,"-37,563.9900","1,604.3601",2020-09-18,0.0012,0.0310,-0.1111,95,156,101,352
116,"1,545.0000","1,980.0000","1,350.0000",-0.6617,0.0001,0.0024,0.7090,0.5479,49.9601,"-38,503.9900","1,594.9601",2020-09-18,0.0013,0.0211,-0.1078,87,147,116,350
95,"1,580.0000","2,000.0000","1,345.0000",-0.6083,0.0000,0.0012,0.4549,0.5479,36.2601,"-38,373.9900","1,616.2601",2020-09-18,0.0009,0.0442,-0.1111,89,165,95,349
120,"1,560.0000","2,000.0000","1,350.0000",-0.6435,0.0000,0.0041,0.5080,0.5479,43.8601,"-39,613.9900","1,603.8601",2020-09-18,0.0011,0.0310,-0.1078,66,158,120,344
155,"1,545.0000","2,000.0000","1,365.0000",-0.6841,0.0001,0.0007,0.7988,0.5479,47.0601,"-40,793.9900","1,592.0601",2020-09-18,0.0012,0.0211,-0.0979,41,145,155,341
84,"1,560.0000","1,980.0000","1,340.0000",-0.6283,0.0000,0.0044,0.4641,0.5479,45.5601,"-37,443.9900","1,605.5601",2020-09-18,0.0012,0.0310,-0.1144,97,157,84,338


In [24]:
potential_ups[(potential_ups.MaxLoss > -10000) &
              (potential_ups.Long_Call_Strike < 460)].sort_values('Short_Put_Strike')

,Short_Put_Strike,Long_Put_Strike,Long_Call_Strike,Delta,Gamma,Theta,Vega,time_remaining,Credit,MaxLoss,BreakEven,Maturity,ReturnOnRisk,downmove_to_breach,upmove_to_win,MaxLossScore,DownMoveScore,UpMoveScore,Score
5,46.0000,28.0000,85.0000,0.2382,0.0006,0.0001,0.0045,0.3148,0.5201,"-1,747.9900",45.4799,2020-07-17,0.0003,-0.2304,0.4221,6,8,5,19
4,46.0000,27.0000,85.0000,0.2394,0.0004,0.0002,0.0043,0.3148,0.5201,"-1,847.9900",45.4799,2020-07-17,0.0003,-0.2304,0.4221,5,7,4,16
3,46.0000,26.0000,85.0000,0.2408,0.0002,0.0004,0.0041,0.3148,0.5451,"-1,945.4900",45.4549,2020-07-17,0.0003,-0.2304,0.4221,3,6,3,12
2,46.0000,25.0000,85.0000,0.2422,0.0001,0.0006,0.0034,0.3148,1.0551,"-1,994.4900",44.9449,2020-07-17,0.0005,-0.2304,0.4221,2,5,2,9
1,47.0000,32.0000,85.0000,0.2467,0.0005,0.0000,0.0099,0.3148,0.6451,"-1,435.4900",46.3549,2020-07-17,0.0004,-0.2137,0.4221,8,4,1,13
0,47.0000,31.0000,85.0000,0.2492,0.0001,0.0003,0.0096,0.3148,0.6901,"-1,530.9900",46.3099,2020-07-17,0.0005,-0.2137,0.4221,7,3,0,10
8,49.0000,25.0000,80.0000,0.3462,0.0005,0.0002,0.0028,0.3148,1.2851,"-2,271.4900",47.7149,2020-07-17,0.0006,-0.1802,0.3385,0,2,8,10
7,50.0000,30.0000,80.0000,0.3583,0.0003,0.0000,0.0085,0.3148,0.9951,"-1,900.4900",49.0049,2020-07-17,0.0005,-0.1635,0.3385,4,1,7,12
6,50.0000,29.0000,80.0000,0.3607,0.0000,0.0003,0.0034,0.3148,1.0051,"-1,999.4900",48.9949,2020-07-17,0.0005,-0.1635,0.3385,1,0,6,7


In [65]:
ib.disconnect()